<a href="https://colab.research.google.com/github/Markus-Heidemann/CarND-Advanced-Lane-Lines/blob/master/IPioneer_Wide_based_stereo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPioneer: Wide-based Stereo

In [1]:
import cv2
import numpy as np
import urllib
import json
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [30, 15]
plt.rcParams['figure.dpi'] = 100

# Data import

In [2]:
def loadImage(img_url):
  resp = urllib.request.urlopen(img_url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  return image

In [3]:
def loadJSON(json_url):
  resp = urllib.request.urlopen(json_url)
  data = json.loads(resp.read().decode())
  return data

In [4]:
img_left_folder_url = "https://raw.githubusercontent.com/Markus-Heidemann/a2d2_datasamples/master/20180810_150607/camera/camera_front_left/20180810150607_camera_frontleft_000000083"
img_right_folder_url = "https://raw.githubusercontent.com/Markus-Heidemann/a2d2_datasamples/master/20180810_150607/camera/camera_front_right/20180810150607_camera_frontright_000000083"

img_left_url = img_left_folder_url + ".png"
img_right_url = img_right_folder_url + ".png"

calib_data_url = "https://raw.githubusercontent.com/Markus-Heidemann/a2d2_datasamples/master/cams_lidars.json"

In [ ]:
img_left = loadImage(img_left_url)
img_right = loadImage(img_right_url)

imgrgb_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2RGB)
imgrgb_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2RGB)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(imgrgb_left)
ax2.imshow(imgrgb_right)

In [6]:
calib_data = loadJSON(calib_data_url)

cam_mat_left = np.array(calib_data["cameras"]["front_left"]["CamMatrix"])
cam_dist_left = np.array(calib_data["cameras"]["front_left"]["Distortion"])

cam_mat_right = np.array(calib_data["cameras"]["front_right"]["CamMatrix"])
cam_dist_right = np.array(calib_data["cameras"]["front_right"]["Distortion"])

# Image undistortion

In [ ]:
d = 90
sigma_color = 100
sigma_space = 100
bilateral_img_left = cv2.bilateralFilter(img_left, d, sigma_color, sigma_space)
bilateral_img_right = cv2.bilateralFilter(img_right, d, sigma_color, sigma_space)
cv2.bilateralFilter?
grayimg_left = cv2.cvtColor(bilateral_img_left, cv2.COLOR_BGR2GRAY)
grayimg_right = cv2.cvtColor(bilateral_img_right, cv2.COLOR_BGR2GRAY)

undst_left_img = cv2.undistort(grayimg_left, cam_mat_left, cam_dist_left)
undst_right_img = cv2.undistort(grayimg_right, cam_mat_right, cam_dist_right)

# smooth_sigma = 3
# smoothed_undst_left_img = cv2.GaussianBlur(undst_left_img, (0, 0), smooth_sigma)
# smoothed_undst_right_img = cv2.GaussianBlur(undst_right_img, (0, 0), smooth_sigma)

In [ ]:
fig = plt.subplots(2, 2)

plt.subplot(3,2,1)
plt.imshow(imgrgb_left)
plt.subplot(3,2,2)
plt.imshow(imgrgb_right)
plt.subplot(3,2,3)
plt.imshow(undst_left_img, cmap='gray', vmin=0, vmax=255)
plt.subplot(3,2,4)
plt.imshow(undst_right_img, cmap='gray', vmin=0, vmax=255)
plt.subplot(3,2,5)
bilateral_img_left_rgb = cv2.cvtColor(bilateral_img_left, cv2.COLOR_BGR2RGB)
plt.imshow(bilateral_img_left_rgb, cmap='gray', vmin=0, vmax=255)
plt.subplot(3,2,6)
bilateral_img_right_rgb = cv2.cvtColor(bilateral_img_right, cv2.COLOR_BGR2RGB)
plt.imshow(bilateral_img_right_rgb, cmap='gray', vmin=0, vmax=255)

# Creation of Disparity Map

In [ ]:
stereo = cv2.StereoSGBM_create(numDisparities=256, blockSize=15)
disparity = stereo.compute(undst_left_img, undst_right_img)

min_disp = np.min(disparity)
max_disp = np.max(disparity)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(disparity, cmap='gray', vmin=0, vmax=max_disp)
ax2.imshow(imgrgb_left)